# Mixture of Gaussian Experiments

In [ ]:
import tensorflow as tf
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

# utility func

In [ ]:
def save_plot(x, epoch=None):
    # make directory if there is not
    path = "gaussian"
    if not os.path.isdir(path):
        os.makedirs(path)    
    
    plt.figure(figsize=(5,5))
    plt.scatter(x[:,0],x[:,1],s=2, color=matplotlib.colors.cnames["orangered"])
    #plt.set_xlabel("x")
    #plt.set_ylabel("y")
    plt.ylim(-2.0, 2.0)
    plt.xlim(-2.0, 2.0)
    plt.legend()
    plt.savefig(os.path.join(path, "dloss" + str(epoch) + ".png"))
    plt.close()

In [ ]:
# save metrics
def save_metrics(metrics, epoch=None):
    # make directory if there is not
    path = "metrics"
    if not os.path.isdir(path):
        os.makedirs(path)

    # save metrics
    plt.figure(figsize=(10,8))
    plt.plot(metrics["dis_loss"], label="discriminative loss", color="b")
    plt.legend()
    plt.savefig(os.path.join(path, "dloss" + str(epoch) + ".png"))
    plt.close()

    plt.figure(figsize=(10,8))
    plt.plot(metrics["gen_loss"], label="generative loss", color="r")
    plt.legend()
    plt.savefig(os.path.join(path, "g_loss" + str(epoch) + ".png"))
    plt.close()

    plt.figure(figsize=(10,8))
    plt.plot(metrics["gen_loss"], label="generative loss", color="r")
    plt.plot(metrics["dis_loss"], label="discriminative loss", color="b")
    plt.legend()
    plt.savefig(os.path.join(path, "both_loss" + str(epoch) + ".png"))
    plt.close()

In [ ]:
# plot images
def save_imgs(images, plot_dim=(5,12), size=(12,5), epoch=None):
    # make directory if there is not
    path = "generated_figures"
    if not os.path.isdir(path):
        os.makedirs(path)

    num_examples = plot_dim[0]*plot_dim[1]
    num_examples = 60
    fig = plt.figure(figsize=size)

    for i in range(num_examples):
        plt.subplot(plot_dim[0], plot_dim[1], i+1)
        img = images[i, :]
        img = img.reshape((96, 96, 3))
        plt.tight_layout()
        plt.imshow(img)
        plt.axis("off")
    plt.subplots_adjust(wspace=0.1, hspace=0.1)
    plt.savefig(os.path.join(path, str(epoch) + ".png"))
    plt.close()

In [ ]:
def fc(x, output_shape, name):
    shape = x.get_shape().as_list()
    dim = np.prod(shape[1:])
    x = tf.reshape(x, [-1, dim])
    input_shape = dim

    initializer = tf.random_normal_initializer(0., 0.001)
    with tf.variable_scope(name):
        weight = tf.get_variable("weight", shape=[input_shape, output_shape], initializer=initializer)
        x = tf.matmul(x, weight)

        bias = tf.get_variable("bias", shape=[output_shape], initializer=tf.constant_initializer(0.))
        x = tf.nn.bias_add(x, bias)

    return x

In [ ]:
import math
def gaussian_mixture_circle(batch_size, num_cluster=8, scale=1, std=2):
    rand_indices = np.random.randint(0, num_cluster, size=batch_size)
    base_angle = math.pi * 2 / num_cluster
    angle = rand_indices * base_angle - math.pi / 2
    mean = np.zeros((batch_size, 2), dtype=np.float32)
    mean[:, 0] = np.cos(angle) * scale
    mean[:, 1] = np.sin(angle) * scale
    return np.random.normal(mean, std**2, (batch_size, 2)).astype(np.float32)

# model

In [ ]:
class Generator:
    def __init__(self):
        self.reuse = False
        self.z_dim = 100
        self.h_dim = 128
        self.o_dim = 2
        self.initializer = tf.contrib.layers.xavier_initializer()

    def __call__(self, x, training=False):
        with tf.variable_scope('g', reuse=self.reuse):
            with tf.variable_scope("fc1"):
                weight = tf.get_variable("W", shape=[self.z_dim, self.h_dim], initializer=self.initializer)
                bias = tf.get_variable("b", shape=[self.h_dim], initializer=self.initializer)
                x = tf.matmul(x, weight) + bias
                x = tf.nn.relu(x)

            with tf.variable_scope("fc2"):
                weight = tf.get_variable("W", shape=[self.h_dim, self.o_dim], initializer=self.initializer)
                bias = tf.get_variable("b", shape=[self.o_dim], initializer=self.initializer)
                x = tf.matmul(x, weight) + bias

        self.reuse = True
        self.variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='g')
        return x

In [ ]:
class Discriminator:
    def __init__(self, batch_size=32):
        self.reuse = False
#        self.initializer = tf.truncated_normal(stddev=0.8)
        self.initializer = tf.contrib.layers.xavier_initializer()
        self.X_dim = 2
        self.o_dim = 1
        
    def __call__(self, x, training=False, name=''):
        with tf.name_scope('d' + name), tf.variable_scope('d', reuse=self.reuse):
            x = x * 0.25

            with tf.variable_scope("fc1"):
                weight = tf.get_variable("W", shape=[self.X_dim, self.o_dim], initializer=self.initializer)
                bias = tf.get_variable("b", shape=[self.o_dim], initializer=self.initializer)
                x = tf.matmul(x, weight) + bias
                x = tf.nn.relu(x)

        self.reuse = True
        self.variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='d')
        return x

In [ ]:
class GAN:
    def __init__(self, batch_size=64):
        self.batch_size = batch_size
        self.g = Generator()
        self.d = Discriminator()
        self.z_dim = 100
        self.batch_size = batch_size
        self.epoch_save_gauss = 1000
        self.epoch_save_metrics = 5000
        self.losses = {"dis_loss":[], "gen_loss":[]}
        
    def loss(self, y, y_gen):
        #d_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=y, labels=tf.ones_like(y))) + tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=y_gen, labels=tf.zeros_like(y_gen))) # 怪しい
        #g_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=y_gen, labels=tf.ones_like(y_gen)))    # 怪しい
        d_loss = -tf.reduce_mean(tf.log(y) + tf.log(1. - y_gen)) 
        g_loss = -tf.reduce_mean(tf.log(y_gen))
        return g_loss, d_loss

    def train(self, batch_size=64, epochs=100000):
        z = tf.placeholder(tf.float32, shape=[None, self.z_dim])
        x = tf.placeholder(tf.float32, shape=[None, 2])
        x_gen = self.g(z)
        y = self.d(x)
        y_gen = self.d(x_gen)
        g_loss, d_loss = self.loss(y, y_gen)
        
        #t_vars = tf.global_variables()
        #g_vars = [var for var in t_vars if "g" in var.name]
        #d_vars = [var for var in t_vars if "d" in var.name]

        opt_d = tf.train.AdamOptimizer(1e-4, beta1=0.5).minimize(d_loss, var_list=self.d.variables)
        opt_g = tf.train.AdamOptimizer(1e-4, beta1=0.5).minimize(g_loss, var_list=self.g.variables)

        z_fix = np.random.uniform(-1, 1, size=[1000, self.z_dim])
        
        self.sess = tf.Session()
        self.sess.run(tf.global_variables_initializer())
        for epoch in range(epochs):
            # 潜在変数と生成データ
            z_gen = np.random.uniform(-1, 1, size=[self.batch_size, self.z_dim])
            
            # 訓練データを生成
            X_tr = gaussian_mixture_circle(self.batch_size, num_cluster=8, scale=2, std=0.1)

            _, dis_loss = self.sess.run([opt_d, d_loss], feed_dict={z: z_gen, x: X_tr})
            _, gen_loss = self.sess.run([opt_g, g_loss], feed_dict={z: z_gen, x: X_tr})

            self.losses["dis_loss"].append(dis_loss)
            self.losses["gen_loss"].append(gen_loss)
            
            if epoch % self.epoch_save_gauss == 0:
                x_ = self.sess.run(x_gen, feed_dict={z: z_fix})
                save_plot(x_, epoch)
                save_plot(X_tr, "epoch" + str(epoch))
            
            if epoch % self.epoch_save_metrics == 0:
                save_metrics(self.losses, epoch)
            
    def sample_images(self, row=5, col=12, inputs=None, epoch=None):
        images = self.g(inputs, training=True)
        return images

In [ ]:
if __name__ == "__main__":
    gan = GAN()
    gan.train()
#    gan.g()